<a href="https://colab.research.google.com/github/mohithxoxo/NLP/blob/master/BERT_with_ktrain(Sentimental_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KTrain is a wrapper for BERT technique of text classification

In [ ]:
!pip install ktrain

In [ ]:
from ktrain import text 
import ktrain

In [ ]:
train = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
test = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/test.tsv', delimiter='\t', header=None)

Extracting training and validation dataset from module

In [ ]:
train = train.rename(columns={0:'text',1:'label'})
test = test.rename(columns={0:'text',1:'label'})

In [ ]:
trn, val, preproc = text.texts_from_df(train,'text', label_columns=['label'], val_df=test,preprocess_mode='bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


Fetching the model and learner instance

In [ ]:
model = text.text_classifier('bert', trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 400
done.


Start to train the model

In [ ]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
1154/1154 [==============================] - 1690s 1s/step - loss: 0.4045 - accuracy: 0.8082 - val_loss: 0.2369 - val_accuracy: 0.9066
Epoch 2/5
1154/1154 [==============================] - 1688s 1s/step - loss: 0.2164 - accuracy: 0.9160 - val_loss: 0.2657 - val_accuracy: 0.8929
Epoch 3/5
1154/1154 [==============================] - 1687s 1s/step - loss: 0.1092 - accuracy: 0.9630 - val_loss: 0.2289 - val_accuracy: 0.9176
Epoch 4/5
1154/1154 [==============================] - 1677s 1s/step - loss: 0.0393 - accuracy: 0.9870 - val_loss: 0.3188 - val_accuracy: 0.9187
Epoch 5/5
1154/1154 [==============================] - 1682s 1s/step - loss: 0.0101 - accuracy: 0.9970 - val_loss: 0.3752 - val_accuracy: 0.9121


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['not_label', 'label']

In [ ]:
#Positive
predictor.predict('happy')

'label'

In [ ]:
#Negative
predictor.predict('sad')

'not_label'

In [ ]:
# let's save the predictor for later use
predictor.save('/content/my_model')
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/content/my_model')
# make a prediction on the same document to verify it still works
reloaded_predictor.predict('sad')